## Solution to task 3

## Quantum Circuit Simulator

In [1]:
import numpy as np

In [2]:
def get_ground_state(num_qubits):
    # return vector of size 2**num_qubits with all zeroes except first element which is 1
    state = np.zeros(2**num_qubits)
    state[0] = 1
    return state

In [3]:
def get_operator(total_qubits, gate_unitary, target_qubits):
    # return unitary operator of size 2**n x 2**n for given gate and target qubits
    
    # Define 2x2 Identity

    I = np.identity(2)
    
    # Define projection operator |0><0|

    P0x0 = np.array([
    [1, 0],
    [0, 0]
    ])

    # Define projection operator |1><1|

    P1x1 = np.array([
    [0, 0],
    [0, 1]
    ])
    
    # Define X gate (CNOT is controlled-X)
    X = np.array([
    [0, 1],
    [1, 0]
    ])
    
    if gate_unitary == 'x':
        if target_qubits[0] == 0:
            O = X
        else:
            O = I
        for i in range(1,total_qubits):
            if i == target_qubits[0]:
                O = np.kron(O,X)
            else:
                O = np.kron(O,I)
    elif gate_unitary == 'h':
    # Define H (Hadamard) gate:
        H = np.array([
        [1/np.sqrt(2), 1/np.sqrt(2)],
        [1/np.sqrt(2), -1/np.sqrt(2)]
        ])
        if target_qubits[0] == 0:
            O = H
        else:
            O = I
        for i in range(1, total_qubits):
            if i == target_qubits[0]:
                O = np.kron(O,H)
            else:
                O = np.kron(O,I)
    # Define CNOT gate            
    elif gate_unitary == 'cx':
        O = [0]*2
        if target_qubits[0] == 0:
            O[0] = P0x0
            O[1] = P1x1
        elif target_qubits[1] == 0:
            O[0] = I
            O[1] = X
        else:
            O[0] = I
            O[1] = I
        for i in range(1, total_qubits):
            if i == target_qubits[0]:
                O[0] = np.kron(O[0], P0x0)
                O[1] = np.kron(O[1], P1x1)
            elif i == target_qubits[1]:
                O[0] = np.kron(O[0], I)
                O[1] = np.kron(O[1], X)
            else:
                O[0] = np.kron(O[0], I)
                O[1] = np.kron(O[1], I)
        O = sum(O)

    
    return O

In [4]:
def run_program(initial_state, program):
    # read program, and for each gate:
    #   - calculate matrix operator
    #   - multiply state with operator
    # return final state
    Nqubits = int(np.log2(len(initial_state)))
    final_state = initial_state
    for gate in program:
        O = get_operator(Nqubits, gate["gate"], gate["target"])
        final_state = np.dot(O, final_state)
        
    return final_state

In [5]:
from numpy.random import choice 
def measure_all(state_vector):
    # choose element from state_vector using weighted random and return it's index
    
    states = list(range(len(state_vector)))
    probs = list(np.abs(state_vector)**2)
    result = choice(states, p=probs) 
    
    return result

In [6]:
def get_counts(state_vector, num_shots):
    # simply execute measure_all in a loop num_shots times and
    # return object with statistics in following form:
    #   {
    #      element_index: number_of_ocurrences,
    #      element_index: number_of_ocurrences,
    #      element_index: number_of_ocurrences,
    #      ...
    #   }
    # (only for elements which occoured - returned from measure_all)

    measurements = {}
    Nstates = len(state_vector)
    Nqubits = int(np.log2(Nstates))
    
    probs = list(np.abs(state_vector)**2)
    for i in range(num_shots):
        result = measure_all(state_vector)
        index = bin(result)[2:]
        index = '0'*(Nqubits-len(index)) + index
        if index not in measurements:
            measurements[index] = 0
        measurements[index] += 1 
    
    measurements = {key: value for key, value in sorted(measurements.items())}
    return measurements

In [9]:
# Define program:

my_circuit = [
{ "gate": "h", "target": [0] }, 
{ "gate": "cx", "target": [0, 1] }
]


# Create "quantum computer" with 2 qubits (this is actually just a vector :) )

my_qpu = get_ground_state(2)


# Run circuit

final_state = run_program(my_qpu, my_circuit)


# Read results

counts = get_counts(final_state, 1000)

print(counts)

# Should print something like:
# {
#   "00": 502,
#   "11": 498
# }

# Voila!


{'00': 485, '11': 515}
